## Labelling
This notebook manages the labelling of the webcam images saved in a goole drive directory.

### Preparations

In [6]:
# import the modules
import os
from os import listdir
from skimage import io #Testkommentar für GIT
import matplotlib.pyplot as plt
import pandas as pd
from time import sleep
from IPython.display import clear_output

# define paths:
master_dir = ""
originals_dir = "Originals/"


# initialize variables:
not_readable_images = []

### Import labelling.csv

Always import the labelling.csv at the beginning so that we import already labelled images.

In [7]:
label_df_from_csv = pd.read_csv(master_dir+"labels.csv")
label_dict = {}

for ind in label_df_from_csv.index:
  wind_direction_csv = label_df_from_csv['wind_direction'][ind]
  wind_force_csv = label_df_from_csv['wind_force'][ind]
  label_list_csv = [wind_direction_csv,wind_force_csv]
  label_dict[label_df_from_csv['Unnamed: 0'][ind]] = label_list_csv

In [ ]:
# Show already labelled items:
countLabels = 0
for k, v in label_dict.items():
  if v != ['windDirectionDummy', 'WindForceDummy']:
    countLabels += 1
    print(k, v)
print(f"Already labelled images: {countLabels}")

In [ ]:
# load new image names to dict and add dummy labels
label_list = ['windDirectionDummy', 'WindForceDummy']
count = 0
for images in os.listdir(originals_dir):
    if (images.endswith(".jpg")) and images not in label_dict:
      label_dict[images] = label_list
      count += 1
print(f'Added items: {count}')

###Manage not readable images (done for first load of images)

In [ ]:
#get not readable images (in case image was not fully downloaded)
for k,v in label_dict.items():
  try:
    image = io.imread(originals_dir+k)
  except:
    not_readable_images.append(k)

In [ ]:
not_readable_images

In [ ]:
 # delete not readable images
print(f"Attention! This will remove the images in not_readable_images from google drive! Number of Images: {len(not_readable_images)}")
print("do you want to proceed? Enter yes..")
answer = input()
if answer == 'yes':
  for image in not_readable_images:
    try:
      os.remove(originals_dir+image)
    except:
      print("Error while deleting file : ", originals_dir+image)
  print("all files deleted!")
else:
  print("no files deleted")


### Labelling

Define the labelling() function. The labels are stored in a dictionary. Make sure to export to CSV from time to time (compare next step).

In [ ]:
#Problem: after imshow the input() doesn't work
#Workaround: use the clear_output() function to clear the image and do the labelling after that
#implement code for inputs like (again and quit)
#define the labels for wind direction and wind force (how many labels?)

#Maybe think about skipping the very cloudy and rainy images (no weather for flying or hiking..).

#do the labelling for not yet labelled items
def labelling():
  maxCount = 10
  count = 0
  sleep_seconds = 7

  for k,v in label_dict.items():
    if v == ['windDirectionDummy', 'WindForceDummy'] and count < maxCount and k not in not_readable_images:
      count += 1
      print(f"Image: {k} - is loading..")
      image = io.imread(originals_dir+k)
      repeat = True
      while repeat:
        fig = plt.figure(1,figsize= (40,15))
        chart0 = fig.add_subplot(121)
        chart1 = fig.add_subplot(122)
        #plt.imshow(image)
        chart0.imshow(image)
        #chart1.imshow(image[220:350,550:700,:]) #vor 28. September 2022
        chart1.imshow(image[180:320,470:600,:]) #ab 28. September 2022
        plt.show()
        sleep(sleep_seconds)
        clear_output()
        print(f'Flag showing to ? for {k}: n,l,r,u,d,0')
        #l - flag showing to the left (wind coming from the right..)
        #r - flag showing to the right
        #u - flag showing up
        #d - flag showing down
        #n - no wind (no direction)
        #0 - not defined (when it's hard to say..)
        #again - show the image again DOES NOT WORK!
        input_wind_direction = input() #To do: make sure only valid categories will be safed
        if input_wind_direction != 'again': #does not work! (image does not show again..)
          repeat = False
      print(f'Enter wind force for {k}: (n,w,m,s,0') #To do: make sure only valid categories will be safed
      #n - no wind (flag down)
      #w - weak wind (flag up to 45 degrees)
      #m - middle wind (flag from 45 degrees to horizontal)
      #s - strong wind (flag horizontal)
      #0 - not defined (when it's hard to say..)
      input_wind_force = input()
      label_dict[k] = [input_wind_direction, input_wind_force]
      print(f"Labels for {k} saved to label_dict.")
  print(f"{count} labels entered - please consider saving the results to labels.csv from time to time..")

In [ ]:
labelling()

### Save results to labels.csv

In [ ]:
column_names = ['wind_direction','wind_force']
label_df_from_dict = pd.DataFrame.from_dict(data=label_dict, orient='index',columns=column_names)
label_df_from_dict.to_csv(master_dir+"labels.csv")